In [ ]:
# %load_ext autoreload
%autoreload 2
import torch
from datasets import TripletAudio

K, MAX_CLOSE_NEG, MAX_FAR_NEG = 5, 15, 15
BATCH_SIZE = 1

triplet_train_dataset = TripletAudio(True, K, MAX_CLOSE_NEG, MAX_FAR_NEG)
triplet_test_dataset = TripletAudio(False, K, MAX_CLOSE_NEG, MAX_FAR_NEG)
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
%autoreload 2
# Set up the network and training parameters
from networks import AnchorNet, EmbeddingNet, TripletNet
from losses import TripletLoss
import torch.optim as optim
from torch.optim import lr_scheduler

INPUT_D, OUTPUT_D = 192, 128
MARGIN, LEARNING_RATE, N_EPOCHS, LOG_INT = 0.5, 1e-3, 5, 1000

#define model
anchor_net = AnchorNet(triplet_train_dataset.get_dataset(), INPUT_D, OUTPUT_D)
embedding_net = EmbeddingNet(anchor_net)
model = TripletNet(embedding_net)

loss_fn = TripletLoss(MARGIN)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)

In [ ]:
%autoreload 2
from trainer import fit
fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, N_EPOCHS, {}, LOG_INT)

### Online Selection

In [27]:
# %load_ext autoreload
%autoreload 2
import torch
from datasets import AudioTrainDataset, AudioTestDataset
from datasets import BalancedBatchSampler

K = 5
train_dataset = AudioTrainDataset(K)
test_dataset = AudioTestDataset(K)

train_batch_sampler = BalancedBatchSampler(train_dataset)
test_batch_sampler = BalancedBatchSampler(test_dataset)

online_train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_batch_sampler)
online_test_loader = torch.utils.data.DataLoader(test_dataset, batch_sampler=test_batch_sampler)

In [44]:
# Set up the network and training parameters
from networks import EmbeddingNet, AnchorNet
from losses import OnlineTripletLoss
from utils import SemihardNegativeTripletSelector, HardestNegativeTripletSelector
from metrics import AverageNonzeroTripletsMetric
import torch.optim as optim
from torch.optim import lr_scheduler

INPUT_D, OUTPUT_D = 192, 128
MARGIN, LEARNING_RATE, N_EPOCHS, LOG_INT = 0.5, 1e-3, 1, 100

#define model
anchor_net = AnchorNet(train_dataset.data, INPUT_D, OUTPUT_D)
model = EmbeddingNet(anchor_net)
loss_fn = OnlineTripletLoss(MARGIN, SemihardNegativeTripletSelector(MARGIN, train_dataset.KNN))
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)

initialising model biases
done
generating similarity matrix
done


In [47]:
%autoreload 2
from trainer import fit
fit(online_train_loader, online_test_loader, model, loss_fn, optimizer, scheduler, N_EPOCHS, {}, LOG_INT, metrics=[AverageNonzeroTripletsMetric()])

Train: [0/48048 (0%)]	Loss: 0.340762	Average nonzero triplets: 132.0
Epoch: 1/5. Train set: Average loss: 0.3408	Average nonzero triplets: 132.0
Epoch: 1/5. Validation set: Average loss: 0.0159	Average nonzero triplets: 7.0
Train: [0/48048 (0%)]	Loss: 0.315916	Average nonzero triplets: 163.0
Epoch: 2/5. Train set: Average loss: 0.3159	Average nonzero triplets: 163.0
Epoch: 2/5. Validation set: Average loss: 0.0023	Average nonzero triplets: 1.0
Train: [0/48048 (0%)]	Loss: 0.285390	Average nonzero triplets: 154.0
Epoch: 3/5. Train set: Average loss: 0.2854	Average nonzero triplets: 154.0
Epoch: 3/5. Validation set: Average loss: 0.0147	Average nonzero triplets: 3.0
Train: [0/48048 (0%)]	Loss: 0.297295	Average nonzero triplets: 132.0
Epoch: 4/5. Train set: Average loss: 0.2973	Average nonzero triplets: 132.0
Epoch: 4/5. Validation set: Average loss: 0.0144	Average nonzero triplets: 2.0
Train: [0/48048 (0%)]	Loss: 0.287615	Average nonzero triplets: 168.0
Epoch: 5/5. Train set: Average loss